# DEEP LEARNING P1
### Name: Aniket Raut
### Roll no. : 22
### Batch : B2


In [10]:
pip install ydata-profiling

## Importing Libraries

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from ydata_profiling import ProfileReport

## Reading Dataset

In [12]:
df = pd.read_csv('/content/Insurance.csv')
df

,person_id,age,sex,bmi,children,smoker,region,total_charges
0,1,19.0,female,27.900,0,yes,southwest,16884.9240
1,2,18.0,male,33.770,1,no,southeast,1725.5522
2,3,28.0,male,33.000,3,no,southeast,4449.4620
3,4,33.0,male,22.705,0,no,northwest,21984.4700
4,5,32.0,male,28.880,0,no,northwest,3866.8552
...,...,...,...,...,...,...,...,...
1333,1334,50.0,male,30.970,3,no,northwest,10600.5480
1334,1335,18.0,female,31.920,0,no,northeast,2205.9807
1335,1336,18.0,female,36.850,0,no,southeast,1629.8335
1336,1337,21.0,female,25.800,0,no,southwest,2007.9450


## Dropping Columns having null Values

In [13]:
df.info()
df = df.dropna(subset=['age', 'bmi'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   person_id      1338 non-null   int64  
 1   age            1337 non-null   float64
 2   sex            1338 non-null   object 
 3   bmi            1334 non-null   float64
 4   children       1338 non-null   int64  
 5   smoker         1337 non-null   object 
 6   region         1332 non-null   object 
 7   total_charges  1338 non-null   float64
dtypes: float64(3), int64(2), object(3)
memory usage: 83.8+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 1333 entries, 0 to 1337
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   person_id      1333 non-null   int64  
 1   age            1333 non-null   float64
 2   sex            1333 non-null   object 
 3   bmi            1333 non-null   float64
 4   children       1333 n

## EDA using Profile Report Library

In [14]:
ProfileReport(df)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

## Applying One Hot Encoding

In [15]:
df = pd.get_dummies(df, columns=['sex', 'smoker', 'region'], drop_first=True)
df

,person_id,age,bmi,children,total_charges,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,1,19.0,27.900,0,16884.9240,False,True,False,False,True
1,2,18.0,33.770,1,1725.5522,True,False,False,True,False
2,3,28.0,33.000,3,4449.4620,True,False,False,True,False
3,4,33.0,22.705,0,21984.4700,True,False,True,False,False
4,5,32.0,28.880,0,3866.8552,True,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...
1333,1334,50.0,30.970,3,10600.5480,True,False,True,False,False
1334,1335,18.0,31.920,0,2205.9807,False,False,False,False,False
1335,1336,18.0,36.850,0,1629.8335,False,False,False,True,False
1336,1337,21.0,25.800,0,2007.9450,False,False,False,False,True


## Spliting Data

In [16]:
X = df.drop('total_charges', axis=1)
y = df['total_charges']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Feature Scaling

In [18]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Function to Create and Train Model

In [19]:
def create_and_train_ann(optimizer, epochs=100, batch_size=32):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))

    if isinstance(optimizer, str):
        optimizer = optimizers[optimizer]
    else:
        optimizer = type(optimizer)(**optimizer.get_config())

    optimizer.clipvalue = 1.0

    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
    return model

## Testing with Diffrent Configurations

In [20]:
optimizers = {
    'SGD': SGD(learning_rate=0.01, momentum=0.0),
    'SGD_momentum': SGD(learning_rate=0.01, momentum=0.9),
    'SGD_nesterov': SGD(learning_rate=0.01, momentum=0.9, nesterov=True),
    'Adam': Adam(learning_rate=0.001),
    'RMSprop': RMSprop(learning_rate=0.001)

}
epochs_list = [10,20,30]
batch_sizes = [16, 32, 64]

results = {}

In [21]:
for opt_name, optimizer in optimizers.items():
  for epochs in epochs_list:
    for batch_size in batch_sizes:

      model = create_and_train_ann(optimizer, epochs, batch_size)
      y_pred = model.predict(X_test)
      mse = mean_squared_error(y_test, y_pred)
      r2 = r2_score(y_test, y_pred)

      results[(opt_name, epochs, batch_size)] = {'mse': mse, 'r2': r2}
      print(f"Optimizer: {opt_name}, Epochs: {epochs}, Batch Size: {batch_size}, MSE: {mse:.2f}, R2: {r2:.2f}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step
Optimizer: SGD, Epochs: 10, Batch Size: 16, MSE: 32774735.46, R2: 0.76


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Optimizer: SGD, Epochs: 10, Batch Size: 32, MSE: 33142535.90, R2: 0.76


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Optimizer: RMSprop, Epochs: 30, Batch Size: 16, MSE: 45333878.01, R2: 0.67


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Optimizer: RMSprop, Epochs: 30, Batch Size: 32, MSE: 225737526.94, R2: -0.63


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Optimizer: RMSprop, Epochs: 30, Batch Size: 64, MSE: 284076870.04, R2: -1.05


## Results

In [22]:
best_model = min(results, key=lambda k: results[k]['mse'])
print(f"\nBest Model: {best_model}, MSE: {results[best_model]['mse']:.2f}, R2: {results[best_model]['r2']:.2f}")



Best Model: ('SGD_nesterov', 20, 16), MSE: 20873425.94, R2: 0.85


## Save the best model

In [23]:
best_model_final = create_and_train_ann(optimizers[best_model[0]], best_model[1],best_model[2])
best_model_final.save('best_insurance_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
